In [11]:

import openturns as ot
import numpy as np
import scipy as sp

class PolynomialChaosExpansion():
  
    def __init__(self, pdf, exp_design_in, exp_design_out):
        
        # case without given pdf: identify PDF from data
        if pdf == None:
            chaos_algo_data = ot.FunctionalChaosAlgorithm(exp_design_in, 
                                                          exp_design_out)
            chaos_algo_data.run()
            self.pdf = chaos_algo_data.getDistribution()
        else:    
            self.pdf = pdf
        self.num_inputs = self.pdf.getDimension()
        self.num_outputs = exp_design_out.shape[1]
        self.num_samples = exp_design_out.shape[0]
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out
        
        # get enumerate function for single and multi-indices
        self.enumerate_function = ot.LinearEnumerateFunction(self.num_inputs)
        
        # assign the correct polynomials based on the input distribution
        self.polynomial_collection = [
                  ot.StandardDistributionPolynomialFactory(
                      self.pdf.getMarginal(i))
                  for i in range(self.num_inputs)]
        
        # get a general product basis, to be used for the construction of 
        # specific PCE bases, i.e., given specific index sets
        self.product_basis = ot.OrthogonalProductPolynomialFactory(
                        self.polynomial_collection, self.enumerate_function)
        
        # get transformation function - very important!
        self.transformation = ot.DistributionTransformation(
                                self.pdf, self.product_basis.getMeasure())
    
    def set_multi_index_set(self, multi_index_set):
        self.multi_index_set = multi_index_set
        self.single_index_set = [self.enumerate_function.inverse(idx) 
                                  for idx in self.multi_index_set]
        
    def set_single_index_set(self, single_index_set):
        self.single_index_set = single_index_set
        self.multi_index_set = [list(self.enumerate_function(idx)) 
                                     for idx in self.single_index_set]
        
    def construct_basis(self):
        self.basis = self.product_basis.getSubBasis(self.single_index_set)
        self.num_polynomials = self.basis.getSize()
        
    def set_exp_design(self, exp_design_in, exp_design_out):
        if self.num_outputs != exp_design_out.shape[1]:
            raise ValueError('Output dimensions do not agree!')
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out 
        self.num_samples = exp_design_out.shape[0]
        
    def evaluate_basis(self, design_in):
        n_samples, n_inputs = np.shape(design_in)
        if n_inputs != self.num_inputs:
            raise ValueError('Input dimensions do not agree!')
        # transform input data
        design_in_tf = np.array(self.transformation(design_in))
        # compute basis evaluation matrix
        eval_matrix = np.array([self.basis[j](design_in_tf) 
                                for j in range(self.num_polynomials)])
        # remove superficial dimension
        eval_matrix = np.squeeze(eval_matrix)
        return eval_matrix.T
    
    def compute_design_matrix(self):
        self.design_matrix = self.evaluate_basis(self.exp_design_inputs)
    
    def compute_coefficients(self):
        # compute design matrix
        self.compute_design_matrix()
        # compute PCE coefficients using least squares regression
        self.coefficients, _, _, singular_values = np.linalg.lstsq(
                                                      self.design_matrix, 
                                                      self.exp_design_outputs,
                                                      rcond=None)
        self.condition_number = singular_values[0] / singular_values[-1]
        print("Koeficienty PCE (prvních pár):", self.coefficients[:5])
        print("Vypočítaná střední hodnota (mean z PCE):", self.compute_mean())
        print("Empirická střední hodnota výstupů:", np.mean(self.exp_design_outputs, axis=0))
    
    def predict(self, design_in):
        eval_matrix = self.evaluate_basis(design_in)
        return eval_matrix.dot(self.coefficients)
    
    def compute_mean(self):
        return self.coefficients[0]
    
    def compute_variance(self):
        return np.sum(np.square(self.coefficients[1:]), axis=0)
        
    
    def compute_sobol_first(self):    
        sobol_f = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        # remove zeroth multi-index from multi-index set
        midx_minus_0 = np.delete(self.multi_index_set, 0, axis=0)
        for i in range(self.num_inputs):
            # remove i-th column from multi-index set without 0
            midx_minus_i = np.delete(midx_minus_0, i, axis=1)
            # get the rows with all indices equal to zero
            row_sum = np.sum(midx_minus_i, axis=1)
            zero_rows = np.asarray(np.where(row_sum==0)).flatten() + 1
            partial_variance = np.sum(np.square(self.coefficients[zero_rows]),
                                      axis=0)
            sobol_f[i,:] = partial_variance / variance
        return sobol_f
    
    def compute_sobol_total(self):
        sobol_t = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        mis = np.array(self.multi_index_set)
        for i in range(self.num_inputs):
            # we want all multi-indices where the i-th index is NOT zero  
            idx_column_i = mis[:,i] 
            non_zero_rows = np.asarray(np.where(idx_column_i!=0)).flatten()
            partial_variance = np.sum(
                                np.square(self.coefficients[non_zero_rows]),
                                axis=0)
            sobol_t[i,:] = partial_variance / variance
        return sobol_t
    
    def compute_generalized_sobol_first(self):
      
        # compute variance and first order Sobol indices (elementwise)
        variance = self.compute_variance()
        sobol_f = self.compute_sobol_first()
        # retrieve elementwise partial variances
        partial_variances = sobol_f*variance
        # compute aggregated variance (scalar value)
        aggregated_variance = np.sum(variance)
        # compute aggregated partial variance per input 
        # 1d array with length equal to num_inputs
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        # compute generalized first order Sobol indices
        sobol_f_gen = aggregated_partial_variances / aggregated_variance
        return sobol_f_gen

    def compute_generalized_sobol_total(self):
   
        # compute variance and total order Sobol indices (elementwise)
        variance = self.compute_variance()
        sobol_t = self.compute_sobol_total()
        # retrieve elementwise partial variances
        partial_variances = sobol_t*variance
        # compute aggregated variance (scalar value)
        aggregated_variance = np.sum(variance)
        # compute aggregated partial variance per input 
        # 1d array with length equal to num_inputs
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        # compute generalized total order Sobol indices
        sobol_t_gen = aggregated_partial_variances / aggregated_variance
        return sobol_t_gen




In [13]:
import openturns as ot
import numpy as np

input_sample = np.array([
    [0.0, 0.0],
    [0.5, 0.2],
    [1.0, 0.4],
    [1.5, 0.6],
    [2.0, 0.8],
    [2.5, 1.0],
    [3.0, 1.2],
    [3.5, 1.4],
    [4.0, 1.6],
    [4.5, 1.8]
])

def test_function(X):
    x1, x2 = X[:, 0], X[:, 1]
    return np.expand_dims(2*x1 + 3*x2**2, axis=1)  # Skalární výstup

output_sample = test_function(input_sample)

# Definice rozdělení vstupů (bez náhodnosti)
marginals = [ot.Uniform(0, 5), ot.Uniform(0, 2)]  # Pevné rozmezí
input_distribution = ot.ComposedDistribution(marginals)

# Inicializace
pce = PolynomialChaosExpansion(input_distribution, input_sample, output_sample)
pce.set_single_index_set([0, 1, 2, 3, 4])  # Jednoduchá indexová množina
pce.construct_basis()
pce.compute_coefficients()

# Predikce na testovacích datech
test_inputs = np.array([
    [0.25, 0.1],
    [0.75, 0.3],
    [1.25, 0.5],
    [1.75, 0.7],
    [2.25, 0.9]
])
predictions = pce.predict(test_inputs)

# Výpočet Sobolových indexů
sobol_first = pce.compute_sobol_first()
sobol_total = pce.compute_sobol_total()

mean_value = pce.compute_mean()
variance_value = pce.compute_variance()

# Výpis výsledků
print("První řád Sobolových indexů:")
print(sobol_first)
print("Celkové Sobolovy indexy:")
print(sobol_total)
print("Střední hodnota modelu:")
print(mean_value)
print("Rozptyl modelu:")
print(variance_value)

true_outputs = test_function(test_inputs)
print("Skutečné hodnoty vs Predikované hodnoty:")
for i in range(len(test_inputs)):
    print(f"Vstupy: {test_inputs[i]}, Skutečná hodnota: {true_outputs[i][0]}, Predikovaná hodnota: {predictions[i][0]}")



Koeficienty PCE (prvních pár): [[ 5.5       ]
 [ 3.17542648]
 [ 3.17542648]
 [-2.23606798]
 [ 3.5       ]]
Vypočítaná střední hodnota (mean z PCE): [5.5]
Empirická střední hodnota výstupů: [7.92]
První řád Sobolových indexů:
[[0.40311804]
 [0.26948775]]
Celkové Sobolovy indexy:
[[0.73051225]
 [0.59688196]]
Střední hodnota modelu:
[5.5]
Rozptyl modelu:
[37.41666667]
Skutečné hodnoty vs Predikované hodnoty:
Vstupy: [0.25 0.1 ], Skutečná hodnota: 0.53, Predikovaná hodnota: 0.529999999999978
Vstupy: [0.75 0.3 ], Skutečná hodnota: 1.77, Predikovaná hodnota: 1.7699999999999854
Vstupy: [1.25 0.5 ], Skutečná hodnota: 3.25, Predikovaná hodnota: 3.2499999999999916
Vstupy: [1.75 0.7 ], Skutečná hodnota: 4.97, Predikovaná hodnota: 4.969999999999995
Vstupy: [2.25 0.9 ], Skutečná hodnota: 6.93, Predikovaná hodnota: 6.929999999999999
